# Generate stores & customers for the demo

## Setup the environment

### Install packages

In [2]:
!pip install faker
!pip install pymorton

### Define environment variables

In [13]:
# Stores
NUMBER_OF_STORES=500
STORES_TABLE='stores'
ACCURACY=20
BASE_ACCURACY=14

# Customers
NUMBER_OF_CUSTOMERS=3000
CUSTOMERS_TABLE='customers'
CUSTOMERS_DATA_FILE='data/trained/data_train_n.csv'

### imports

In [2]:
import logging
import os
import pickle
import random
import itertools

# Demo
import pandas as pd
import pymorton as pm
from faker import Faker
from faker.providers import BaseProvider

# DB
import v3io as v3c
import v3io_frames as v3f

### Open DB client

In [3]:
client = v3f.Client('http://'+os.getenv('V3IO_FRAMESD'))

## Generate Stores

### Location provider

In [215]:
class LocationProvider(BaseProvider):
    '''
    Creates locations within base_location

    Uses QuadTree for Geohashing
        @{http://tech.taskrabbit.com/blog/2015/06/09/elasticsearch-geohash-vs-geotree/}
        @{http://mapzen.github.io/leaflet-spatial-prefix-tree/}
        @{http://blog.notdot.net/2009/11/Damn-Cool-Algorithms-Spatial-indexing-with-Quadtrees-and-Hilbert-Curves}
    '''    
    def location(self, location_base: str, base_acc=10, acc=20):
        coordinates = location_base[:base_acc]
        for i in range(acc-len(coordinates)):
            coordinates += str(random.randint(0, 3))
        return coordinates

Add the location provider as a faker provider

In [216]:
faker = Faker()
faker.add_provider(LocationProvider)

### Define scenario environment (locations)

In [303]:
london_city = (51.514926, -0.089580)
london_city_south = (51.501593, -0.094942)
london_west = (51.512309, -0.128966)
london_south_west = (51.495022, -0.162268)

london_coordinates = []
london_coordinates.append(london_city)
london_coordinates.append(london_city_south)
london_coordinates.append(london_west)
london_coordinates.append(london_south_west)

london_coordinates_qt = list(map(lambda cooridnate: pm.interleave_latlng(*cooridnate), london_coordinates))
london_coordinates_qt

['03311311313011311011000321002320',
 '03311311311233323013031101320003',
 '03311311313010023000032330133111',
 '03311311311222300331010333220231']

### Define Store class

In [309]:
class Store(object):
    def __init__(self, store_id: int, name: str, location: str):
        self.store_id = store_id if store_id is not None else random.randint(1, int(os.genev('NUMBER_OF_STORES', 500)))
        self.location = location
        self.store_name = name
    
    def json(self):
        json_store = {
            'id': self.store_id,
            'name': self.store_name,
            'location': self.location
        }
        return json_store

### Create stores

In [310]:
def create_stores(faker, number_of_stores: int=500):
    remaining = number_of_stores
    stores = []
    while remaining > 0:
        # Create stores
        stores_batch = [Store(faker.msisdn(), faker.company(), faker.location(random.choice(london_coordinates_qt), BASE_ACCURACY, ACCURACY)).json() 
              for i in range(remaining)]
        
        # Count for duplicate locations
        locations = list(map(lambda store: store['location'], stores_batch))
        locations = list(dict.fromkeys(locations))
        remaining -= len(locations)
        
        # Update stores list
        stores.append(stores_batch)

    stores = list(itertools.chain.from_iterable(stores))
    stores = pd.DataFrame.from_records(stores)
    stores = stores.set_index(['location'])
    stores = stores[~stores.index.duplicated(keep='first')]
    return stores

In [311]:
stores = create_stores(faker, NUMBER_OF_STORES)
stores.head(5)

,id,name
location,,
03311311311233123233,8854466828553,Johnson and Sons
03311311313011222013,1926061225066,Melton-Davidson
03311311313010033230,5091376789245,Turner-Collier
03311311311233030232,6260815076834,Brooks LLC
03311311313011112312,1908495859430,Shelton Inc


### Upload to DB 

In [301]:
# client.delete('kv', STORES_TABLE)
client.write('kv', STORES_TABLE, stores)

{'num_frames': 1, 'num_rows': 500}

In [307]:
client.read('kv', STORES_TABLE).head(5)

,id,name
__name,,
03311311313010021013,2122086612786,Turner and Sons
03311311313011100031,2639533316617,Green PLC
03311311311233302103,9932603130377,"Richardson, Jackson and Norton"
03311311311222000133,3426225259247,"Wong, Sexton and Grimes"
03311311313010232202,9992248366318,"Fisher, Schneider and Singleton"


## Generate customers

In [8]:
class Customer(object):
    def __init__(self, id=0, location='', products=""):
        self.id = id
        self.location = location
        self.products = products
        
    def json(self):
        json_customer = {
            'id': self.id,
            'location': self.location,
            'products': self.products
        }
        return json_customer

In [20]:
def create_customers(customers_data: pd.DataFrame, number_of_customers: int):
    available_customer = list(customers_data.user_id)
    customers = [Customer(id=i,
                          products=customers_data.loc[customers_data['user_id'] == random.choice(available_customer)]\
                                    .drop(['user_id', 'Unnamed: 0'], axis=1)\
                                    .to_json()).json() for i in range(number_of_customers)]
    customers = pd.DataFrame.from_records(customers)
    customers.set_index(['id'])
    return customers

In [21]:
# Load customers seed data
customers_data = pd.read_csv(CUSTOMERS_DATA_FILE)

# Create actual customers
customers = create_customers(customers_data, NUMBER_OF_CUSTOMERS)
customers.head(5)

,id,location,products
0,0,,"{""user_product_reordered_ratio"":{""1803"":1.0,""1..."
1,1,,"{""user_product_reordered_ratio"":{""2851"":1.0},""..."
2,2,,"{""user_product_reordered_ratio"":{""18243"":1.0},..."
3,3,,"{""user_product_reordered_ratio"":{""6335"":1.0},""..."
4,4,,"{""user_product_reordered_ratio"":{""1617"":1.0,""1..."


## Upload to DB

In [22]:
# client.delete('kv', CUSTOMERS_TABLE)
client.write('kv', CUSTOMERS_TABLE, customers)

{'num_frames': 1, 'num_rows': 3000}

In [23]:
client.read('kv', CUSTOMERS_TABLE).head(5)

,id,location,products
__name,,,
2586,2586.0,,"{""user_product_reordered_ratio"":{""1839"":1.0},""..."
1983,1983.0,,"{""user_product_reordered_ratio"":{""10231"":1.0,""..."
1232,1232.0,,"{""user_product_reordered_ratio"":{""566"":1.0,""17..."
939,939.0,,"{""user_product_reordered_ratio"":{""6087"":1.0},""..."
521,521.0,,"{""user_product_reordered_ratio"":{""3774"":1.0},""..."


## Generate predictions

In [15]:
client.create('tsdb', 'predictions', attrs={'rate': '1/s'})